In [1]:
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from math import exp,log
import numpy as np
import copy
import re, string, unicodedata
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.feature_extraction import stop_words
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report,confusion_matrix
import nltk

In [2]:
data = pd.read_csv(r"spam.csv", encoding='latin-1')
# data.drop(data.index[1000:], inplace=True)
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
inp = data.iloc[:,1:2]
outp = data.iloc[:,0:1]
split = .75
train_data, train_label = inp.iloc[:(int)(inp.shape[0]*split),:] , outp.iloc[:(int)(inp.shape[0]*split),:] 
test_data, test_label = inp.iloc[(int)(inp.shape[0]*split):,:] , outp.iloc[(int)(inp.shape[0]*split):,:]
outp.head()

,v1
0,ham
1,ham
2,spam
3,ham
4,ham


In [4]:
lemma = WordNetLemmatizer()
def text_process(mess):
    mess = mess.lower()
    stop_free = " ".join([i for i in mess.split() if i not in stopwords.words('english')])
    nonum = [char for char in mess if char not in ['0','1','2','3','4','5','6','7','8','9']]
    nopunc =[char for char in nonum if char not in string.punctuation]
    nopunc=''.join(nopunc)
    lem = " ".join(lemma.lemmatize(word) for word in nopunc.split())
    return lem

In [5]:
# outp.is_copy = False
inp['v2'] = inp['v2'].apply(text_process)
# outp['v2'][5569:5573]
outp.shape
# outp.head()

C:\Users\Narahc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(5572, 1)

In [6]:
print(inp.head())
nham = (outp.v1 == 'ham').sum()
nspam = (outp.v1 == 'spam').sum()
ntot = outp.shape[0]
print("Ham :", nham)
print("Spam :", nspam)
print("Total :", ntot)

                                                  v2
0  go until jurong point crazy available only in ...
1                            ok lar joking wif u oni
2  free entry in a wkly comp to win fa cup final ...
3        u dun say so early hor u c already then say
4  nah i dont think he go to usf he life around h...
Ham : 4825
Spam : 747
Total : 5572


In [7]:
vectorizer = CountVectorizer()
vectorizer.fit(inp['v2'])
spam_ar = inp[(outp.v1 == 'spam')] 
ham_ar = inp[(outp.v1 == 'ham')]
print(len(spam_ar))
print(len(ham_ar))
# print(vectorizer.get_vocablaray)
spam_vec = vectorizer.transform(spam_ar['v2'])
ham_vec = vectorizer.transform(ham_ar['v2'])
# print(spam_vec)

spam_freq = np.array(spam_vec.sum(axis=0))
ham_freq = np.array(ham_vec.sum(axis=0))

# spam_freq_np = np.array(spam_freq)
# ham_freq_np = np.array(ham_freq)
no_spam = len(spam_ar)
no_ham = len(ham_ar)
spam_freq = spam_freq.reshape(spam_freq.shape[1])
ham_freq = ham_freq.reshape(ham_freq.shape[1])
tot_freq = spam_freq + ham_freq
spam_words = spam_vec.sum() 
ham_words = ham_vec.sum()
tot_words = spam_words + ham_words
print(tot_words)

747
4825
76155


In [8]:
x = 2e-300
y=1
print(x)
print(x>y)
import sys
sys.float_info

2e-300
False


sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

In [9]:
# def predict(predis):
# #     predis = "XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL"
# #     print(text_process(predis))
#     predis = text_process(predis)
#     # prediv = vectorizer.transform([predis])
#     # print(prediv.sum(axis=0))

#     # print(len(vectorizer.vocabulary_))
#     tot_num = 1.00
#     tot_den = 1.00
#     for word in predis.split():
#         if word in vectorizer.vocabulary_:
#     #         print(spam_freq[vectorizer.vocabulary_[word]])
#             tot_num *= ((spam_freq[vectorizer.vocabulary_[word]] + 1) / (spam_words + len(vectorizer.vocabulary_)))
#             tot_den *= tot_freq[vectorizer.vocabulary_[word]] / tot_words
#     spamm = (tot_num * (no_spam / ( no_spam + no_ham)) )/tot_den
    
#     tot_num = 1.00
#     tot_den = 1.00
#     for word in predis.split():
#         if word in vectorizer.vocabulary_:
#     #         print(spam_freq[vectorizer.vocabulary_[word]])
#             tot_num *= ((ham_freq[vectorizer.vocabulary_[word]] + 1) / (ham_words + len(vectorizer.vocabulary_)))
#             tot_den *= tot_freq[vectorizer.vocabulary_[word]] / tot_words
#     hamm = (tot_num * (no_ham / ( no_spam + no_ham)) )/tot_den
#     print(spamm,hamm)
#     if spamm > hamm:
#         return "spam"
#     return "ham"

In [10]:
def predict(predis):
#     predis = "XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL"
#     print(text_process(predis))
    predis = text_process(predis)
    # prediv = vectorizer.transform([predis])
    # print(prediv.sum(axis=0))

    # print(len(vectorizer.vocabulary_))
    tot_num = 1.00
    tot_den = 1.00
    for word in predis.split():
        if word in vectorizer.vocabulary_:
    #         print(spam_freq[vectorizer.vocabulary_[word]])
            tot_num *= ((spam_freq[vectorizer.vocabulary_[word]] + 1) / (spam_words + len(vectorizer.vocabulary_)))
            tot_den *= tot_freq[vectorizer.vocabulary_[word]] / tot_words
    spamm = (tot_num * ((no_spam +1)/ ( no_spam + no_ham + 2)) )/tot_den
    
    tot_num = 1.00
    tot_den = 1.00
    for word in predis.split():
        if word in vectorizer.vocabulary_:
    #         print(spam_freq[vectorizer.vocabulary_[word]])
            tot_num *= ((ham_freq[vectorizer.vocabulary_[word]] + 1) / (ham_words + len(vectorizer.vocabulary_)))
            tot_den *= tot_freq[vectorizer.vocabulary_[word]] / tot_words
    hamm = (tot_num * ((no_ham + 1) / ( no_spam + no_ham +2 )) )/tot_den
    print(spamm,hamm)
    if spamm > hamm:
        return "spam"
    return "ham"

In [11]:
tru = 0
fal = 0
for i in range(inp.shape[0]):
    if predict(inp['v2'][i]) == outp['v1'][i]:
        tru += 1
    else :
        fal += 1
#     print(inp['v2'][i])
#     break
print(tru," ",fal)

2.125147062934856e-05 4.030589374440588
4.797146146204502e-05 2.125478349185319
27262812.241119925 1.1654360606890958e-13
2.559081915095152e-09 2.6775275724843826
7.650391726929377e-11 1.6306695281397459
1.7535790149430078e-06 0.00014430503611137987
3.1000587582150185e-08 2.3705845788174478
0.001230633513124894 0.13485351136032178
729857.0678609262 1.1847843047429305e-15
2063.8485449429413 9.391154015238478e-11
2.1654398167942128e-11 2.234096856324048
784983.2008743484 1.617943777241148e-09
510955.4883117291 2.4448751114771323e-14
2.0911332295256773e-10 1.3532986643514529
0.027406529586028676 0.33374716679381977
546.6998425734563 6.542159501499033e-05
0.0005153807788614888 2.491200809454054
1.866687909693903e-08 1.5714293503848389
4.948362688593702e-08 5.009218274627726
444977.8686248344 9.146699431796462e-06
3.999657487349234e-05 1.2019577135691124
0.002248046295959514 0.30676797814609225
2.7509896632526373e-08 1.975538374288469
6.54712530896874e-16 2.1746951495750424
1.95320328462013

4.791291000629703e-05 0.4605022950719246
2.4704403562142387e-10 2.269502034200596
0.003722709035956221 0.6330469772967245
9.29452145416262e-06 1.6068179945831766
6.0622848999515605e-05 1.4062556832852533
5.956045993325466 1.8463591964175638e-05
4.950582900589659e-07 2.840957823782307
4.2597492862979883e-07 0.2017940169823018
0.00026121880790131493 2.1226998376104724
1.571415315454242e-11 0.2821790142928742
1.977582281782257 1.8669645908969043e-06
0.00016163440303865921 0.6902925456318427
0.037422743155246276 7.364780626510709
3.50569474002538e-08 1.7315181879580195
8.049212509485694e-17 4.911451269252464
0.0005702296667347477 1.7982465251383204
1.304526496322943e-07 0.6649541410324044
0.0008884106252375888 0.19659047884750733
0.014020903955294952 1.1027829583818571
1.192740645060731e-19 2.9063322051110676
2113179.291042864 8.197676892946358e-15
0.07537876966643375 3.8514992189766395
3.940370778154309e-15 11.080020532853368
0.005743577674141889 1.2282027118900305
1.358148695099597e-09 3

5.8089825193109205e-08 29.46665731358406
3.1493937577952165e-09 0.5650215780976547
7.48669506368546e-07 0.4309414484502898
1116598.2527401075 7.980250586473792e-13
9.544874122762263e-07 0.7510036357334434
7.272367656087123e-05 0.8960309248301449
1.9315889079921505e-11 2.4720537366068416
2.1254571187729263e-07 0.9885700564627556
0.0014306922739820776 1.2918890611552736
7.027918617639872e-07 0.4120001702134942
0.00014544836673429526 1.3414801404700794
15052502.430830123 1.8812217130524987e-13
4.3732904841134185e-06 1.3201838363347622
1.275614273846945e-17 0.3886547175176767
16663.805380558253 4.467029445973894e-10
1.2593617810351704e-10 1.124693068564953
2.448378168396699e-08 1.5183941402183485
0.002361799399219507 1.366546783105034
0.20001802324985393 7.769161270041293
1.2534929036256464e-06 1.2453549511264967
2.949872732330281e-07 5.347515514579989
0.0013770679940080207 0.9772270751685326
1.0864639951148671e-10 1.0558174002233176
0.0002266924231843106 0.9784485702946033
1.5738084602716

0.5309961848481342 0.0010880095229463345
0.007601465378987854 1.5371042598827913
2.3726373954769734e-11 1.775552485283751
1.4428411540047163e-09 1.7068383910255267
0.0003245189661286469 0.004404647988667877
2.3995776438536137e-10 1.8425137069562298
0.00021191928905754532 2.454523472582067
1.8876126766331685e-05 1.3135692383614048
1.1078305614549711e-08 1.6108267072345912
1.6479902677629974e-06 1.1345236833904877
0.014368119389490423 0.17548094179180976
0.00021785330338865936 0.9388761762906506
1.5943532097578722e-05 0.504522569494928
0.003755389983614989 5.007687630618628
8.481461962328216e-14 2.2339040674892114
3.184499276086391e-05 8.98529210778584
1.1373649342401507e-07 1.5618529876114033
0.00033062329895795577 2.05860899001198
1.049742088899197e-06 2.752834027087184
14719.584924932713 2.7607855065136405e-08
6514445.062469181 7.922399283250113e-13
1.0440517308110166e-19 336.9916178536064
2.4568031721848192e-18 5.493832818429576
1.3473150497384132 0.023265515205893202
6.4914017674748

3.4835317904240138e-06 1.0037842287533647
7.53638815477279e-07 0.492766100761078
0.0003044144649061064 3.3826446287947713
1.1628187946584141e-05 6.403148952502473
3.836095007371726e-06 0.6653373765112611
1687865480.098416 1.7474528842612276e-05
1.1147263759839922e-09 2.8190675500308697
0.12698164994727176 1.5668972607587928
5877155.9186579045 2.6390117171760557e-11
5.770865791257923e-19 1.2583787989641375
1.7816312283967826e-05 1.109638055306628
8.028123661031572e-14 2.8938382675935252
6.359999785933501e-13 1.6081637841398553
0.0010830603078309854 0.08239457200048811
1.3086366688886537e-07 1.8371891940495662
0.00010178116888061868 0.2766027025969421
6.791191240205481e-13 3.5838708522467573
5.3966062784417904e-08 0.763402705767376
4.087804795524927e-07 0.6488637097442805
5.835338977274588e-05 1.1897786844754572
1.1557646353219553e-05 0.961159579878144
0.00034197919492419935 2.933646505859732
9.99923109771193e-08 4.650125681483781
0.002504847213645801 16.732870960629878
0.001191231702033

...

0.00455096759877508 3.0431986545609435
3.3863676128507055e-08 1.433103835776025
4.164070870331871e-12 4.173763699807091
6.332639241475584e-07 0.7672118160352425
1.4324838278731009e-11 0.05316688632037819
2.5088231503204317 0.4735056624886298
5.406458325991183e-08 1.6773094787794611
0.00012084075246976688 5.463675116582514
3.8062659278128634e-05 1.0926377766329938
1.8349868489644564e-05 1.4454369389043575
7.468905701274635e-17 3.52043102114413
0.000789593461032167 0.5037141742657778
3.389128087505236e-06 5.524245628256542
0.00022585193305474054 0.55970011348856
1.391086101886004e-12 30.7579171458817
1.2347513508354703e-19 1.7895769641112724
7.629772809685373e-09 1.4857706286480048
2.032331423053955e-11 13.720576874323417
1.804928333576375e-05 1.0290346628747995
1901.2441025528076 2.4070773587423915e-08
1.6108594372443599 3.385795196340274
2.3008779867671066e-05 1.6456367159869638
183.12397198706347 1.6411497968348292e-06
0.00041877720090348006 0.6921497844016967
0.0007508645546184232 2.

4.505776354084242e-10 2.1007905088849887
0.003166756339739924 2.420508802951924
6.476195409515797e-05 0.7538547398987675
0.17357067640123217 1.9023606115374774
1.7794604106017702 0.6768851064071073
9.900591390791533e-08 0.03461817706375459
0.4718249691964978 0.4287072446343147
63.01249063246839 0.00010405727382777912
9.33552651279588e-08 1.3266046170558377
0.0011799029426372441 107.94717020540527
1.4589421233262578e-05 12.294169559674959
0.00037310380404906567 2.172735326582966
5.054179106103512e-06 0.0008630883255310112
6.00024691375599e-11 1.5547708087105865
0.003801918435700819 0.5510047494507948
6.12893869215402e-12 1.1783080845392528
2.5089338283412485e-10 3.177768857208708
3.5751049867929134e-06 0.7326717416391109
3.4286407168517776e-11 2.086968717463188
9.260128253496179e-05 1.3345382175039202
2.2197292681308844e-08 1.6833578548539534
1.2388103677939774e-08 3.754139498086142
0.06569775797778951 0.9040858184304157
2.27255354062509e-13 1.3865096250118698
4.473578481393693e-06 0.64

2.0117440165528877e-07 2.3281780847796587
0.000171047032676867 0.1132479185533677
1.0740122740800315e-08 0.32238262754467156
0.00017987776304810162 0.8974440260445204
8.098243933807968e-06 1.2572648748986623
0.0003455653779605886 3.7433468418523175
0.002325633854142938 1.4955046682205022
0.15388236427614307 0.0006751201672517693
0.0005767530526882148 1.7653517215795897
1.5708010491435936e-05 1.2414477304706546
7.903929172605707e-09 3.2354144476234237
0.2225358923595603 0.6160589381809201
6.472760856363185e-07 0.6874865902333291
2224845.4545499515 7.405319803287133e-13
1.0722158673560979e-06 3.3570773596162864
2.5113441811148174e-05 1.2863065797549522
0.007290283123982316 1.1001154466055831
5.766017691771199e-06 0.7518238195064548
4.996593769258241e-06 1.5578612402104601
1.376342267563815e-06 0.6393599350932286
6.406827307221885e-05 1.5740138643386394
0.0007959769178427044 0.7060891318775218
0.0232433385084906 5.076498321611869
5.70112234774616e-07 3.7689512719306224
0.00010101218457345

1.561122203551834e-16 1.0783237505918775
6.427879214271102e-17 2.5845800939742616
2.8282673896028113e-08 1.1328482410026197
0.00015182274316507418 0.9260495698058443
0.22916893806716968 0.9217066878799912
6.410919344300534e-10 1.1740477242382792
0.00727389738785499 3.569963710012961
0.03698731395009608 2.0904795782109766
0.029313597990370033 1.9916003345265647
0.02348378631578195 1.0017999893073635
2277873.474235747 5.165288796673791e-15
15035.37116295572 2.1022770130443463e-11
1.5943532097578722e-05 0.504522569494928
7.468365215804251e-10 1.6724234710007313
0.0018726342948158484 1.4694215017414867
0.00017234101550368922 0.4188515268167227
3.9053627122191926e-22 17.655816207408616
1090821.4133467874 6.798020491875065e-13
2.8318240395841073e-09 1.345079458071417
3.227713862299952e-09 0.5299865240099232
0.00010745690781807237 8.857267546687892
110621.65390578187 6.315322604209033e-06
25088619.61742884 1.256852555461049e-15
0.0057482536522423305 0.2348091501240113
0.0021616856231723015 1.

1.5667817720349525e-08 3.5230858322141727
1.5029789385513557e-06 1.259901495474328
0.011649597030970446 1.257787670804777
0.0007641033266474736 1.6694738578180977
0.0061845693463378655 2.06858638642167
0.0007438704862516726 1.4385600276153452
3.0783340817925615e-10 8.268037568789888
5.733733374716664e-05 1.724201621590656
1.9732543115924087e-07 1.3958713324237437
0.018111990607606474 0.9906376529514209
0.02623844217282234 3.4865041303710123
7.1548721204841365e-06 0.747054527862539
0.004244904490371944 3.9555493668000405
1.6629996392870479e-13 0.7235648372546136
6.786842225920225e-08 8.448025854523674
4.1039600198120144e-17 1.8537225106549937
4.865504687065775e-05 0.4807686148801244
2.4527264862316006e-05 9.46556597359256
1.9692776004441803e-05 1.0818151654202262
0.00013663416474790502 0.8774043231243276
1.694401357000942e-20 3.2806371804154275
0.002673376917201801 0.7397328402389819
0.00022407735047663533 0.7374622015649056
1.0206532022125317e-06 4.527135973088489
1.716715947816474e-07

In [12]:
tru / (tru+fal)

0.9885139985642498